# Tratamento de *missings*

## Carregar Módulos

In [34]:
# geral
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## Carregar dados

In [35]:
dados_hatco = pd.read_csv('hatco_csv.csv', sep=';', decimal=',')

In [36]:
dados_hatco.head()

,id,velocidade_entrega,nivel_preco,flexibilidade_preco,imagem_fornecedor,qualidade_servico,imagem_forca_vendas,qualidade_produto,tamanho_firma,nivel_uso,nivel_satisfacao,area_industria
0,1,4.1,0.6,6.9,4.7,2.4,2.3,5.2,pequena,32.0,4.2,industria
1,2,1.8,3.0,6.3,6.6,2.5,4.0,8.4,grande,43.0,4.3,outra_area
2,3,3.4,5.2,5.7,6.0,4.3,2.7,8.2,grande,48.0,5.2,industria
3,4,2.7,1.0,7.1,5.9,1.8,2.3,7.8,grande,32.0,3.9,industria
4,5,6.0,0.9,9.6,7.8,3.4,4.6,4.5,pequena,58.0,6.8,industria


In [37]:
dados_hatco.shape

(170, 12)

## Análises

### Exemplo 1.1 - verificar os casos incompletos por linha

Obteremos o número de dados perdidos por linha

In [ ]:
# Possibilidades de usar o axis dentro do sum()
#Axis=0: Soma os elementos de cada linha em um DataFrame ou array.
#Axis=1: Soma os elementos de cada coluna em um DataFrame ou array.

In [38]:
# contar os missings por linhas
missings_por_linha = pd.DataFrame({'id': dados_hatco['id'].tolist(),
                                   'n_missings': dados_hatco.isna().sum(axis=1).tolist()})

In [39]:
missings_por_linha

,id,n_missings
0,1,0
1,2,0
2,3,0
3,4,0
4,5,0
...,...,...
165,266,0
166,267,2
167,268,1
168,269,2


Podemos obter o percentual de dados perdidos por linha.

In [40]:
# número total de colunas:
# utilizamos o -1 para retirar a coluna ID, pois ela não é uma pergunta
n_colunas = dados_hatco.shape[1] - 1
n_colunas

11

In [41]:
# A função assign() permite adicionar ou modificar colunas em um DataFrame.
# verificar os resultados, criando uma nova coluna chamada perc_missings no missings_por_linha
missings_por_linha \
    .assign( perc_missings = missings_por_linha['n_missings'] / n_colunas ) \
    .sort_values('perc_missings', ascending = False)  \
    .head(10)

,id,n_missings,perc_missings
162,263,7,0.636364
109,210,7,0.636364
113,214,6,0.545455
160,261,6,0.545455
144,245,5,0.454545
132,233,5,0.454545
103,204,3,0.272727
120,221,3,0.272727
123,224,3,0.272727
106,207,3,0.272727


### Exemplo 1.2 Verificar casos incompletos por coluna

Obter o número de *missings* por coluna

In [42]:
# contar as colunas
missings_por_coluna = dados_hatco.isna().sum()

In [43]:
#criar nova tabela com missings por coluna
missings_por_coluna = pd.DataFrame( missings_por_coluna,
                                   columns=['n']) \
                                   .reset_index() \
                                   .rename(columns={'index': 'variaveis'})

In [44]:
missings_por_coluna

,variaveis,n
0,id,0
1,velocidade_entrega,21
2,nivel_preco,13
3,flexibilidade_preco,17
4,imagem_fornecedor,7
5,qualidade_servico,9
6,imagem_forca_vendas,6
7,qualidade_produto,9
8,tamanho_firma,2
9,nivel_uso,9


Obter o percentual

In [45]:
n_linhas = dados_hatco.shape[0]
n_linhas

170

In [46]:
# verificar os resultados
missings_por_coluna \
    .assign( perc_missings = missings_por_coluna['n'] / n_linhas) \
    .sort_values('n', ascending = False)

,variaveis,n,perc_missings
1,velocidade_entrega,21,0.123529
3,flexibilidade_preco,17,0.100000
2,nivel_preco,13,0.076471
5,qualidade_servico,9,0.052941
7,qualidade_produto,9,0.052941
9,nivel_uso,9,0.052941
4,imagem_fornecedor,7,0.041176
10,nivel_satisfacao,7,0.041176
6,imagem_forca_vendas,6,0.035294
8,tamanho_firma,2,0.011765


### Identificação dos *missings*

- Método 1: comparando variáveis qualitativas

a) Contagem das variáveis qualitativas

In [47]:
dados_hatco['tamanho_firma'] \
      .value_counts(dropna=False) \
      .to_frame('n') \
      .reset_index() \
      .rename(columns={'index': 'tamanho_firma'})

,tamanho_firma,n
0,pequena,103
1,grande,65
2,NaN,2


b) Contagem  da tabela cruzada entre variáveis qualitatitivas

In [48]:
# cruzamento das variáveis
pd.crosstab(dados_hatco.tamanho_firma.fillna('NA'),
            dados_hatco.area_industria.fillna('NA'))

area_industria,NA,industria,outra_area
tamanho_firma,,,
NA,0,0,2
grande,1,32,32
pequena,0,51,52


c) Correlação das variáveis quantitativas

In [49]:
# filtar os missings
dados_hatco[['velocidade_entrega', 'qualidade_produto']].corr()

,velocidade_entrega,qualidade_produto
velocidade_entrega,1.000000,-0.404352
qualidade_produto,-0.404352,1.000000


In [50]:
# atribuir 0 aos missings
dados_hatco[['velocidade_entrega', 'qualidade_produto']].fillna(0).corr()

,velocidade_entrega,qualidade_produto
velocidade_entrega,1.00000,-0.12319
qualidade_produto,-0.12319,1.00000


### Tratamento dos *missings*

- Eliminar todos os *missings*

In [51]:
dados_hatco.shape

(170, 12)

In [52]:
dados_hatco \
    .dropna() \
    .shape

(126, 12)

- Imputar pelo dados mais frequentes

In [53]:
dados_hatco['area_industria'] \
      .value_counts(dropna=False) \
      .to_frame('n') \
      .reset_index() \
      .rename(columns={'index': 'area_industria'})

,area_industria,n
0,outra_area,86
1,industria,83
2,NaN,1


In [54]:
# aqui passa o valor nulo para ['outra_area']

dados_hatco['area_industria'] \
      .fillna('outra_area') \
      .value_counts(dropna=False) \
      .to_frame('n') \
      .reset_index() \
      .rename(columns={'index': 'area_industria'})

,area_industria,n
0,outra_area,87
1,industria,83


- Atribuição média/mediana

In [55]:
stats = dados_hatco[['flexibilidade_preco', 'imagem_fornecedor']].describe()

In [56]:
stats

,flexibilidade_preco,imagem_fornecedor
count,153.000000,163.000000
mean,7.952288,5.217178
std,1.391378,1.144099
min,5.000000,2.500000
25%,6.700000,4.500000
50%,8.200000,5.000000
75%,9.200000,6.000000
max,10.000000,8.200000


In [57]:
stats['flexibilidade_preco'].loc['mean']

7.952287581699348

In [58]:
stats['flexibilidade_preco'].loc['50%']

8.2

In [59]:
dados_hatco[['id', 'flexibilidade_preco', 'imagem_fornecedor']].tail(5)

,id,flexibilidade_preco,imagem_fornecedor
165,266,9.4,4.9
166,267,NaN,NaN
167,268,9.7,3.3
168,269,NaN,4.5
169,270,8.7,4.6


In [60]:
dados_hatco2 = dados_hatco \
    .assign(flexibilidade_preco2 = dados_hatco['flexibilidade_preco'].fillna(stats['flexibilidade_preco'].loc['50%']) ) \
    .assign(imagem_fornecedor2 = dados_hatco['imagem_fornecedor'].fillna(stats['imagem_fornecedor'].loc['mean']) )

In [61]:
dados_hatco2[['id', 'flexibilidade_preco', 'flexibilidade_preco2', 'imagem_fornecedor', 'imagem_fornecedor2']].tail(5)

,id,flexibilidade_preco,flexibilidade_preco2,imagem_fornecedor,imagem_fornecedor2
165,266,9.4,9.4,4.9,4.900000
166,267,NaN,8.2,NaN,5.217178
167,268,9.7,9.7,3.3,3.300000
168,269,NaN,8.2,4.5,4.500000
169,270,8.7,8.7,4.6,4.600000


In [62]:
dados_hatco2[['flexibilidade_preco', 'flexibilidade_preco2', 'imagem_fornecedor', 'imagem_fornecedor2']].describe()

,flexibilidade_preco,flexibilidade_preco2,imagem_fornecedor,imagem_fornecedor2
count,153.000000,170.000000,163.000000,170.000000
mean,7.952288,7.977059,5.217178,5.217178
std,1.391378,1.321646,1.144099,1.120154
min,5.000000,5.000000,2.500000,2.500000
25%,6.700000,6.900000,4.500000,4.600000
50%,8.200000,8.200000,5.000000,5.100000
75%,9.200000,9.100000,6.000000,5.900000
max,10.000000,10.000000,8.200000,8.200000
